# Ollama web summarizer 

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown

In [2]:
class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)

        # parse the HTML content of the webpage
        soup = BeautifulSoup(response.content, 'html.parser')

        # If a <title> tag exists, soup.title.string fetches its text content. 
        # If the title is missing, it assigns the string "No title found" to self.title.
        self.title = soup.title.string if soup.title else "No title found"

        # This loop iterates over and finds all <script>, <style>, <img>, and <input> elements within the <body> of the HTML.
        # decompose() permanently removes these elements from the soup object to ensure that non-essential content doesn't clutter the text.
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        self.text = soup.body.get_text(separator="\n", strip=True)

In [3]:
system_prompt = "You are an assistant that analyzes the contents of a website \
                and provides a short summary, ignoring text that might be navigation related. \
                Respond in markdown."

In [4]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [5]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [6]:
import ollama

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model='llama3.2', 
        messages = messages_for(website)
    )
    return response['message']['content']

In [7]:
web_summary = summarize("https://link.springer.com/chapter/10.1007/978-981-15-7219-7_22")

In [8]:
Markdown(web_summary)

Here is the paper summary in a shorter format:

**Title**: Convolutional Neural Network Based Chest X-Ray Image Classification for Pneumonia Diagnosis.

**Authors:** Rushi Bhatt, Sudhanshu sinh Yadav, and Jignesh N Sarvaiya.

**Journal:** Communications in Computer and Information Science, Springe, Singapore, and the S. National Institute of Technology.

The study examines a machine learning approach to classify pneumonia on chest X-Ray images using the Convolutional Neural Network.

**Abstract:**

Pneumonia can be an acute infectious lung disease that can cause significant mortality rates globally, especially in low-income regions, the dataset was collected for developing predictive models. 

This research uses data and dataset from the original source with permission. We thank the authors of [15] for sharing their results.

In this paper study we explore using a Machine learning algorithm to develop a Convolutional neural network based model for classification, chest X-ray images of Pneumonia diagnosis by applying CNN architecture for classifying image features learned from deep learning techniques using pre-trained models and fine tuning the trained model by removing noise.
Keywords: 
Convolutional Neural Networks (CNN) and Image Classfication  of 

Jignesh Sarvaiya
View author publications
You can also search for this author in

 PubMed